In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

engine = create_engine(
    "mssql+pyodbc://DESKTOP-MHNR294\\SQLEXPRESS/sales_db"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)


In [5]:

ORDER_ID_START = 1004
DETAIL_ID_START = 3002

PROMO_START = datetime(2024,6,1)
PROMO_END   = datetime(2024,6,15)
PROMO_DISCOUNT = 0.20  
PROMO_PRODUCTS = [1, 3, 6]  


In [6]:
products = pd.read_sql("SELECT product_id, price FROM Products", engine)
customers = pd.read_sql("SELECT customer_id FROM Customers", engine)

prod_price = dict(zip(products.product_id, products.price))
cust_ids = customers.customer_id.tolist()


In [7]:
def rand_date(start, end):
    return start + timedelta(days=random.randint(0, (end-start).days))


In [ ]:
orders = []
oid = ORDER_ID_START

plan = [
    ("before", 1200, datetime(2024,4,1), datetime(2024,5,31)),
    ("during", 1800, PROMO_START, PROMO_END),
    ("after",  1000, datetime(2024,6,16), datetime(2024,7,15)),
]

for label, cnt, d1, d2 in plan:
    for _ in range(cnt):
        orders.append({
            "order_id": oid,
            "customer_id": random.choice(cust_ids),
            "order_date": rand_date(d1, d2),
            "total_amount": 0.0,
            "period": label
        })
        oid += 1

orders_df = pd.DataFrame(orders)
orders_df_sql = orders_df.drop(columns=["period"])

orders_df_sql.to_sql(
    "Orders",
    engine,
    schema="dbo",
    if_exists="append",
    index=False
)


print("Orders inserted:", len(orders_df))


In [12]:
details = []
did = DETAIL_ID_START

for _, o in orders_df.iterrows():
    items = random.randint(1, 4)
    chosen = random.sample(list(prod_price.keys()), items)

    for pid in chosen:
        qty = random.randint(2, 5) if o.period == "during" else random.randint(1, 3)
        price = prod_price[pid]

        if o.period == "during" and pid in PROMO_PRODUCTS:
            price = round(price * (1 - PROMO_DISCOUNT), 2)

        details.append({
            "order_detail_id": did,
            "order_id": o.order_id,
            "product_id": pid,
            "quantity": qty,
            "unit_price": price
        })
        did += 1

details_df = pd.DataFrame(details)
details_df.to_sql("Order_Details", engine, schema="dbo", if_exists="append", index=False)

print("Order_Details inserted:", len(details_df))


Order_Details inserted: 10146


In [13]:
with engine.begin() as conn:
    conn.execute(text("""
        UPDATE Orders
        SET total_amount = (
            SELECT SUM(quantity * unit_price)
            FROM Order_Details od
            WHERE od.order_id = Orders.order_id
        )
        WHERE order_id >= :start
    """), {"start": ORDER_ID_START})

print("Totals updated")


Totals updated
